In [1]:
import pickle
import data.read_data as read
import data.transform_data as transform
from tqdm import tqdm
import torch as th


In [2]:
model = pickle.load(open("saved/conv_3/model_150k_lowercase_12h_12m_35s_18_01_2019.p", "rb"))
vocab = pickle.load(open("saved/conv_3/vocab_150k_lowercase_12h_12m_35s_18_01_2019.p", "rb"))
count = pickle.load(open("saved/conv_3/count_150k_lowercase_12h_12m_35s_18_01_2019.p", "rb"))


In [3]:
df = read.load_train_csv("res/test.csv")


In [4]:
print(df.iloc[0:2, 0])


0    00001cee341fdb12
1    0000247867823ef7
Name: id, dtype: object


In [5]:
print(df.iloc[0:2, 1])


0    Yo bitch Ja Rule is more succesful then you'll...
1    == From RfC == \n\n The title is fine as it is...
Name: comment_text, dtype: object


In [6]:
idx_and_sent = transform.split_comment(df.iloc[:, :].values)


In [7]:
print(idx_and_sent[0:2])

[('00001cee341fdb12', array(['yo', 'bitch', 'ja', 'rule', 'is', 'more', 'succesful', 'then',
       'you', 'll', 'ever', 'be', 'whats', 'up', 'with', 'you', 'and',
       'hating', 'you', 'sad', 'mofuckas', 'i', 'should', 'bitch', 'slap',
       'ur', 'pethedic', 'white', 'faces', 'and', 'get', 'you', 'to',
       'kiss', 'my', 'ass', 'you', 'guys', 'sicken', 'me', 'ja', 'rule',
       'is', 'about', 'pride', 'in', 'da', 'music', 'man', 'dont', 'diss',
       'that', 'shit', 'on', 'him', 'and', 'nothin', 'is', 'wrong',
       'bein', 'like', 'tupac', 'he', 'was', 'a', 'brother', 'too',
       'fuckin', 'white', 'boys', 'get', 'things', 'right', 'next',
       'time'], dtype='<U9')), ('0000247867823ef7', array(['from', 'rfc', 'the', 'title', 'is', 'fine', 'as', 'it', 'is',
       'imo'], dtype='<U5'))]


In [8]:
idx, sentence = transform.pass_data_to_word_idx(idx_and_sent, vocab, count)


In [9]:
padded_sentence = transform.pad_sentences(sentence, 1400, vocab[transform.padding])


In [10]:
batch_size = 32
nb_batch = int(padded_sentence.shape[0] / batch_size)
print(nb_batch)


4786


In [11]:
submission_name = "first_submission_conv_150k.csv"
file = open(submission_name, "w")
file.write("id,toxic,severe_toxic,obscene,threat,insult,identity_hate\n")


58

In [12]:
model.eval()
model.cuda()

for i in tqdm(range(nb_batch)):
    i_min = i * batch_size
    i_max = (i + 1) * batch_size
    i_max = i_max if i_max < padded_sentence.shape[0] else padded_sentence.shape[0]
    
    idx_batch = idx[i_min:i_max]
    sentence_batch = th.Tensor(padded_sentence[i_min:i_max]).long().cuda()
    
    res = model(sentence_batch).detach().cpu().numpy()
    
    to_write = ""
    for j in range(res.shape[0]):
        to_write += "{},{},{},{},{},{},{}\n".format(idx_batch[j], 
                                                    res[j, 0], res[j, 1], res[j, 2], res[j, 3], res[j, 4], res[j, 5])
    file.write(to_write)


  0%|          | 0/4786 [00:00<?, ?it/s]

  0%|          | 1/4786 [00:00<26:29,  3.01it/s]

  1%|          | 26/4786 [00:00<18:32,  4.28it/s]

  1%|          | 54/4786 [00:00<12:59,  6.07it/s]

  2%|▏         | 84/4786 [00:00<09:06,  8.60it/s]

...

 98%|█████████▊| 4671/4786 [00:14<00:00, 327.77it/s]

 98%|█████████▊| 4704/4786 [00:14<00:00, 325.49it/s]

 99%|█████████▉| 4737/4786 [00:15<00:00, 321.29it/s]

100%|█████████▉| 4771/4786 [00:15<00:00, 324.46it/s]

100%|██████████| 4786/4786 [00:15<00:00, 315.32it/s]

In [13]:
file.close()
